<div align="center">
<p align="center" style="width: 100%;">
    <img src="https://raw.githubusercontent.com/vlm-run/.github/refs/heads/main/profile/assets/vlm-black.svg" alt="VLM Run Logo" width="80" style="margin-bottom: -5px; color: #2e3138; vertical-align: middle; padding-right: 5px;"><br>
</p>
<p align="center"><a href="https://docs.vlm.run"><b>Website</b></a> | <a href="https://docs.vlm.run/"><b>API Docs</b></a> | <a href="https://docs.vlm.run/blog"><b>Blog</b></a> | <a href="https://discord.gg/AMApC2UzVY"><b>Discord</b></a>
</p>
<p align="center">
<a href="https://discord.gg/AMApC2UzVY"><img alt="Discord" src="https://img.shields.io/badge/discord-chat-purple?color=%235765F2&label=discord&logo=discord"></a>
<a href="https://twitter.com/vlmrun"><img alt="Twitter Follow" src="https://img.shields.io/twitter/follow/vlmrun.svg?style=social&logo=twitter"></a>
</p>
</div>

Welcome to **[VLM Run Cookbooks](https://github.com/vlm-run/vlmrun-cookbook)**, a comprehensive collection of examples and notebooks demonstrating the power of structured visual understanding using the [VLM Run Platform](https://app.vlm.run). 

## Advanced Video Transcription with VLM Run

This notebook demonstrates how to extract both audio transcripts and visual scene descriptions from video content using VLM Run's advanced video transcription capabilities.

### Environment Setup

To get started, install the VLM Run Python SDK and sign-up for an API key on the [VLM Run App](https://app.vlm.run).
- Store the VLM Run API key under the `VLM_RUN_API_KEY` environment variable.

### Prerequisites

* Python 3.9+
* VLM Run API key (get one at [app.vlm.run](https://app.vlm.run))

## Setup

First, let's install the required packages:

In [2]:
! pip install "vlmrun[all]" --quiet
! pip install yt-dlp --quiet

## Configure VLM Run

In [3]:
import os
import getpass

VLMRUN_BASE_URL = os.getenv("VLMRUN_BASE_URL", "https://dev.vlm.run/v1")
VLMRUN_API_KEY = os.getenv("VLMRUN_API_KEY", None)
if VLMRUN_API_KEY is None:
    VLMRUN_API_KEY = getpass.getpass()

 ········


In [4]:
from vlmrun.client import VLMRun

client = VLMRun(base_url=VLMRUN_BASE_URL, api_key=VLMRUN_API_KEY)

### Download sample YouTube video

For this example, we're going to be using a sample YouTube video.


In [5]:
# Download sample youtube video for transcription purposes
import yt_dlp
from vlmrun.constants import VLMRUN_TMP_DIR

URL = "https://www.youtube.com/watch?v=KxjPgGLVJSg"

height = 720
options = {
    "outtmpl": str(VLMRUN_TMP_DIR / "%(id)s.%(ext)s"),
    "format": f"bestvideo[height<={height}][ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best",
    "keepvideo": True,
}
with yt_dlp.YoutubeDL(options) as ydl:
    info = ydl.extract_info(URL, download=True)
    path = VLMRUN_TMP_DIR / f"{info['id']}.mp4"
print(f"Downloaded video [path={path.name}, size={path.stat().st_size / 1024 / 1024:.2f} MB]")

[youtube] Extracting URL: https://www.youtube.com/watch?v=KxjPgGLVJSg
[youtube] KxjPgGLVJSg: Downloading webpage
[youtube] KxjPgGLVJSg: Downloading tv client config
[youtube] KxjPgGLVJSg: Downloading player b21600d5
[youtube] KxjPgGLVJSg: Downloading tv player API JSON
[youtube] KxjPgGLVJSg: Downloading ios player API JSON
[youtube] KxjPgGLVJSg: Downloading m3u8 information
[info] KxjPgGLVJSg: Downloading 1 format(s): 398+140
[download] Destination: /Users/kaushikbokka/.vlmrun/tmp/KxjPgGLVJSg.f398.mp4
[download] 100% of   17.14MiB in 00:00:02 at 5.75MiB/s   
[download] Destination: /Users/kaushikbokka/.vlmrun/tmp/KxjPgGLVJSg.f140.m4a
[download] 100% of    7.65MiB in 00:00:02 at 2.74MiB/s   
[Merger] Merging formats into "/Users/kaushikbokka/.vlmrun/tmp/KxjPgGLVJSg.mp4"
Downloaded video [path=KxjPgGLVJSg.mp4, size=24.85 MB]


### Visualize the video

In [8]:
from IPython.display import HTML, display

_, yt_id = URL.split("?v=")
IFRAME_STR = f'<iframe width="560" height="315" src="https://www.youtube.com/embed/{yt_id}?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>'

display(HTML(IFRAME_STR))

### Generate structured data from a long-form video

Let's take this 4-minute long video and generate audio and visual trascripts. We take both the audio and video transcripts and segment them into ~20s scenes.

In [9]:
from vlmrun.client.types import GenerationConfig

# Generate structured data from the video
response = client.video.generate(
    domain="video.transcription",
    file=path,
    batch=True,
    config=GenerationConfig(detail="hi"),
)
print(response.model_dump_json(indent=2))

2025-03-12 12:27:43.724 | DEBUG    | vlmrun.client.predictions:_handle_file_or_url:317 - Uploading file [path=/Users/kaushikbokka/.vlmrun/tmp/KxjPgGLVJSg.mp4, size=24.85 MB] to VLM Run
2025-03-12 12:27:43.726 | DEBUG    | vlmrun.client.files:get_cached_file:56 - Computing md5 hash for file [file=/Users/kaushikbokka/.vlmrun/tmp/KxjPgGLVJSg.mp4]
2025-03-12 12:27:43.793 | DEBUG    | vlmrun.client.files:get_cached_file:62 - Computed md5 hash for file [file=/Users/kaushikbokka/.vlmrun/tmp/KxjPgGLVJSg.mp4, hash=8e8ee35999cc6b6a45a6ed3f9dfac24a]
2025-03-12 12:27:43.793 | DEBUG    | vlmrun.client.files:get_cached_file:65 - Checking if file exists in the database [file=/Users/kaushikbokka/.vlmrun/tmp/KxjPgGLVJSg.mp4, hash=8e8ee35999cc6b6a45a6ed3f9dfac24a]
2025-03-12 12:27:45.066 | DEBUG    | vlmrun.client.files:get_cached_file:78 - File hash does not exist in the database [file=/Users/kaushikbokka/.vlmrun/tmp/KxjPgGLVJSg.mp4, hash=8e8ee35999cc6b6a45a6ed3f9dfac24a, exc=404 Client Error: Not Foun

{
  "id": "160484ff-8229-4a10-a2ff-62c8929f5146",
  "created_at": "2025-03-12T06:57:53.930184",
  "completed_at": null,
  "response": null,
  "status": "pending",
  "usage": {
    "elements_processed": null,
    "element_type": null,
    "credits_used": null
  }
}


In [11]:
from vlmrun.client.types import PredictionResponse

# Wait for the prediction to complete
response: PredictionResponse = client.predictions.wait(id=response.id, timeout=1000, sleep=5)
assert isinstance(response, PredictionResponse)

Waiting for prediction to complete:  36%|██████▊            | 358/1000 [3:40:42<6:35:47, 36.99s/it]


APIError: HTTPSConnectionPool(host='dev.vlm.run', port=443): Read timed out. (read timeout=120)

### Analyzing the Transcription Results

The transcription result contains rich structured data with both audio and visual information for each segment. Let's explore different ways to visualize and work with this data:

#### 1. Understanding the Response Structure

The response contains:
- `segments`: List of video segments with audio and visual transcriptions
- `metadata`: Overall video information (language, content, topics, duration)

In [43]:
import pandas as pd
pd.set_option('display.max_colwidth', 80)

# Print the high-level video transcription
df = pd.json_normalize(response.response)
df.head()

,segments,metadata.language,metadata.content,metadata.topics,metadata.duration
0,"[{'start_time': 0.0, 'end_time': 25.8, 'audio': {'content': ' Like the only ...",None,None,None,488.56


#### 2. Exploring Segment Details

Each segment contains:
- `start_time` and `end_time`: Temporal boundaries in seconds
- `audio.content`: Text transcription of spoken content
- `video.content`: Description of visual elements in the scene

In [40]:
pd.set_option('display.max_colwidth', 600)

segments_json = response.response.get("segments", [])
segments_df = pd.json_normalize(segments_json)
segments_df["preview"] = segments_df.apply(
    lambda x: IFRAME_STR.replace("?rel=0", f"?start={int(x['start_time'])}&end={int(x['end_time'])}"), axis=1
)
HTML(segments_df.to_html(escape=False))

,start_time,end_time,audio.content,video.content,preview
0,0.00,25.80,"Like the only way to find these opportunities to learn about them is to find weirdos on the internet that are also into this thing. Yes. And they're figuring it out too. And you can kind of compare notes. Yes. And this is how new industries are created. Literally. By weirdos on the internet. Like literally. Literally. This is Dalton, plus Michael, and today we're going to talk about why AI is going to create more successful founders in the world.","Two men are engaged in a conversation at a table. The man on the left, wearing a light gray shirt, is gesturing with his hands as he speaks. The man on the right, dressed in a blue shirt, listens attentively and occasionally responds with hand gestures. They appear to be in a professional setting, possibly an office or conference room, with large windows in the background allowing natural light to fill the space.",
1,25.80,51.71,"It's interesting, as we've gotten older, we kind of see a new set of tools come into the market and then an explosion in the number of founders who can now create value. And we've seen this before, right? Like, what was the first time you saw this? I certainly noticed when the internet was new, people that knew how to build websites were suddenly able to make lots of money from","The video features two individuals engaged in a conversation at a table. The person on the left, wearing a light gray shirt, is facing the person on the right, who is dressed in a blue jacket over a black shirt. The background is minimalistic, with a plain wall and a window allowing natural light to enter. The text overlay on the left side of the screen reads ""AI Will Create More Successful Founders"" and ""Founder Explosion."" On the right side, there is a list titled ""Founder Explosion"" with various items such as ""On The Cusp,"" ""Cost Of Business,"" ""Get In Early,"" ""Whatnot,"" ""Endless Opportunity,"" and ""Internet Weirdos."" The conversation appears to be focused on the impact of artificial intelligence on business and entrepreneurship.",
2,51.71,71.89,"the skill. And it was like really basic stuff. High school kids were making tons of money. Yep. I remember people that could just figure out how to sell stuff on eBay, where you would go buy something cheap but then listed on eBay and arbitrage. Yep. Basically, you would see people that kind of understood the new tooling that came out and would like do a hustle and make ungodly amounts of money.","The video features two men engaged in a conversation in an office setting. The man on the left, wearing a light gray button-up shirt, is actively speaking and gesturing with his hands, while the man on the right, dressed in a blue jacket over a black shirt, listens attentively with his arms crossed. The background includes a large window with blinds partially drawn, allowing natural light to filter into the room. The conversation appears to be focused on business-related topics, as indicated by the text on the right side of the screen, which lists various themes such as 'On The Cusp,' 'Cost Of Business,' 'Get In Early,' 'Whatnot,' 'Endless Opportunity,' and 'Internet Weirdos.' The overall atmosphere suggests a professional discussion.",
3,72.01,92.67,"Yeah. And it was just because they understood the new tools. And I already wasn't even a hustle. Like it was a good business. Like it was, they saw that tools enabled new businesses. You know, we saw this, you know, tail end of the open source world where like we could build all of Justin TV with free software. Yep.","The video features two men engaged in a conversation in an office setting. The man on the left, wearing a light gray shirt, is gesturing animatedly with his hands as he speaks, indicating an active discussion. The man on the right, dressed in a blue jacket over a black shirt, listens attentively with his hands clasped together on the table. The background includes a window with blinds partially op

As you can see, the video has been segmented into ~20s scenes each with detailed audio transcriptions and corresponding visual captions. This provides developers with a powerful means to understand the video content at a granular level.

### Thanks for following along!

Head over to the [VLM Run App](https://app.vlm.run) to try out the [VLM Run](https://vlm.run) API for yourself!